### 크롬 드라이버 창 열기

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--mute-audio")  # 음소거 옵션 추가

driver = webdriver.Chrome(options=options)

---

### 로그인 시도

In [ ]:
# import undetected_chromedriver as uc

# driver = uc.Chrome()
# driver.get("https://youtube.com")

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import global_constants

# 로그인이 완료되면 유튜브로 넘어가는 url
login_url = "https://accounts.google.com/v3/signin/identifier?checkedDomains=youtube&continue=https%3A%2F%2Fwww.youtube.com%2Fsignin%3Faction_handle_signin%3Dtrue%26app%3Ddesktop%26hl%3Dko%26next%3Dhttps%253A%252F%252Fwww.youtube.com%252F&ddm=1&ec=65620&flowEntry=ServiceLogin&flowName=GlifWebSignIn&hl=ko&ifkv=AcMMx-fxZ82P_dh4l0x5OjBskffPj3UKT340XP69FSCdZDGoG_c21nyWoAp-KMGFQaoU1LvrcTn1_Q&pstMsg=1&service=youtube&dsh=S136025753%3A1730345560396742"
driver.get(login_url)

# 아이디 입력
username_input = driver.find_element(By.CSS_SELECTOR, "input#identifierId.whsOnd.zHQkBf")
username_input.send_keys(global_constants.my_youtube_id)
username_input.send_keys(Keys.RETURN)

In [3]:
# 비밀번호 입력
password_input = driver.find_element(By.CSS_SELECTOR, "input.whsOnd.zHQkBf")
password_input.send_keys(global_constants.my_youtube_pw)
password_input.send_keys(Keys.RETURN)

로그인 완료!

---

### 키워드 서치

In [ ]:
# 빈도분석을 바탕으로 선정한 서치 키워드
search_keyword_list = [
    "Kim+Jong-un+health+problem+death",
    "Kim+Jong-un+health+disappearance",
    "Kim+Jong-un+health+crisis",
    "Kim+Jong-un+missing",
    "Kim+Jong-un+critical+condition"
]

In [ ]:
import time
import pandas as pd

video_urls = []

for search_keyword in search_keyword_list:
    # 키워드 검색하기
    url = f"https://www.youtube.com/results?search_query={search_keyword}+after%3A2020-04-11+before%3A2020-05-01"

    driver.get(url)
    print(f"search keyword: {search_keyword}")

    # 스크롤 다섯 번 내리기
    for _ in range (5):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)

    # 검색된 영상들의 url 수집하기
    video_renderer_tags = driver.find_elements(By.CSS_SELECTOR, "ytd-video-renderer.style-scope.ytd-item-section-renderer")

    for video_renderer_tag in video_renderer_tags:
        video_urls.append([search_keyword, video_renderer_tag.find_element(By.CSS_SELECTOR, "a#video-title").get_attribute('href')]) 

# 수집한 url을 csv 파일로 저장
df = pd.DataFrame(video_urls, columns=["search_keyword", "video_url"])
df.to_csv("video_urls.csv", index=False)
print("video_urls.csv file saved!")

스크롤 10번 내렸더니 쌩뚱맞은 영상들이 너무 많이 나와서, 일단 5번만 내려서 수집하였음.

601행 * 2열 ["search_keyword", "video_url”]

---

### 동영상 정보 수집하기

In [5]:
import pandas as pd
import time

df = pd.read_csv("video_urls.csv")  # video_urls.csv 파일을 데이터프레임으로 불러오기 
video_urls = df['video_url'].tolist()  # video_url 칼럼을 리스트로 가져오기
video_urls = list(set(video_urls))  # 중복 제거

full_video_data_list = []
error_video_urls = []

for i, video_url in enumerate(video_urls[307:]):
    driver.get(video_url)
    print(f"\n\n\n{i+307}")

    time.sleep(10)
    
    # "관련 동영상" 버튼 클릭
    try:
        related_video_button = driver.find_element(By.CSS_SELECTOR, 'yt-formatted-string.style-scope.yt-chip-cloud-chip-renderer[title="관련 콘텐츠"]')
        related_video_button.click()
    except Exception as e:
        pass

    # discription text "...더보기" 버튼 클릭
    try:
        more_discription_button = driver.find_element(By.CSS_SELECTOR, "tp-yt-paper-button#expand.button.style-scope.ytd-text-inline-expander")
        more_discription_button.click()
    except Exception as e:
        pass

    # 스크롤 맨 밑으로 내리기 (최대 20회)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    for _ in range (20):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        # 현재 스크롤 높이 확인
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:  # 스크롤을 끝까지 내렸다면 for 루프 종료
            break
        # 스크롤 높이 업데이트
        last_height = new_height

    # 영상 정보 수집
    try:
        # 영상 제목
        video_title = driver.find_element(By.CSS_SELECTOR, "yt-formatted-string.style-scope.ytd-watch-metadata").text
        # 영상 업로더
        video_uploader_name = driver.find_element(By.CSS_SELECTOR, "yt-formatted-string#text.style-scope.ytd-channel-name.complex-string").text
        # 업로더 채널 주소
        video_uploader_url = driver.find_element(By.CSS_SELECTOR, "yt-formatted-string#text.style-scope.ytd-channel-name.complex-string > a").get_attribute('href')
        # 좋아요 수
        likes_count = driver.find_element(By.CSS_SELECTOR, "#top-level-buttons-computed > segmented-like-dislike-button-view-model > yt-smartimation > div > div > like-button-view-model > toggle-button-view-model > button-view-model > button > div.yt-spec-button-shape-next__button-text-content").text
        # 조회 수
        views_count = driver.find_element(By.CSS_SELECTOR, "#info > span:nth-child(1)").text
        # 업로드 날짜
        video_upload_date = driver.find_element(By.CSS_SELECTOR, "#info > span:nth-child(3)").text
        # decription text
        decription_text = driver.find_element(By.CSS_SELECTOR, "#description-inline-expander > yt-attributed-string").text
        # 댓글 수
        try:
            comment_count = driver.find_element(By.CSS_SELECTOR, "#count > yt-formatted-string > span:nth-child(2)").text
        except Exception as e:  # 댓글 사용을 막아놓은 영상의 경우에 대한 예외 처리
            comment_count = None

        video_data_list = [video_url, video_title, video_uploader_name, video_uploader_url, likes_count, views_count, video_upload_date, decription_text, comment_count]
        print("VIDEO INFO: ", video_data_list)
    except Exception as e:
        print("video_data_list ERROR: ", video_url, e)
        error_video_urls.append(video_url)
        continue

    # 댓글 수집
    if comment_count != None:  # 댓글 사용이 허용된 경우에만
        try:
            comment_thread_renderer_tags = driver.find_elements(By.CSS_SELECTOR, "ytd-comment-thread-renderer.style-scope.ytd-item-section-renderer")
            comment_data_list = []
            for comment_thread_renderer_tag in comment_thread_renderer_tags:
                comment_author_name = comment_thread_renderer_tag.find_element(By.CSS_SELECTOR, "a#author-text.yt-simple-endpoint.style-scope.ytd-comment-view-model").text
                comment_author_id = comment_thread_renderer_tag.find_element(By.CSS_SELECTOR, "a#author-text.yt-simple-endpoint.style-scope.ytd-comment-view-model").get_attribute('href')
                comment_date = comment_thread_renderer_tag.find_element(By.CSS_SELECTOR, "span#published-time-text.style-scope.ytd-comment-view-model").text
                comment_text = comment_thread_renderer_tag.find_element(By.CSS_SELECTOR, "span.yt-core-attributed-string.yt-core-attributed-string--white-space-pre-wrap").text
                comment_likes_count = comment_thread_renderer_tag.find_element(By.CSS_SELECTOR, "span#vote-count-middle.style-scope.ytd-comment-engagement-bar").text
                try:                                    
                    replies_count = comment_thread_renderer_tag.find_element(By.CSS_SELECTOR, "#expander > div.expander-header.style-scope.ytd-comment-replies-renderer").text
                except Exception as e:
                    replies_count = None
                    
                comment_data = [comment_author_name, comment_author_id, comment_date, comment_text, comment_likes_count, replies_count]
                comment_data_list.append(comment_data)
                # print("COMMENTS: ", comment_data)
        except Exception as e:
            print("comment_data_list ERROR: ", video_url, e)
            continue
    else:  # 댓글 사용이 막힌 경우 빈 리스트를 반환
        comment_data_list = []
    
    # 관련 동영상 url 수집
    try:
        related_compact_video_tags = driver.find_elements(By.CSS_SELECTOR, "ytd-compact-video-renderer.style-scope.ytd-item-section-renderer")
        related_videos_list = []
        for related_compact_video_tag in related_compact_video_tags:
            related_videos_list.append(related_compact_video_tag.find_element(By.CSS_SELECTOR, "a#thumbnail.yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail").get_attribute('href')) 
        # print("RELATED VIDEOS: ", related_videos_list)
    except Exception as e:
        print("related_videos_list ERROR: ", video_url, e)
        continue

    full_video_data_list.append(video_data_list + [comment_data_list] + [related_videos_list])




307
VIDEO INFO:  ['https://www.youtube.com/watch?v=RO4MSOJqdtw&pp=ygVDS2ltIEpvbmctdW4gaGVhbHRoIHByb2JsZW0gZGVhdGggYWZ0ZXI6MjAyMC0wNC0xMSBiZWZvcmU6MjAyMC0wNS0wMQ%3D%3D', 'Where Is Kim Jong-un? How Experts Track North Korea’s Leader | NYT News', 'The New York Times', 'https://www.youtube.com/@nytimes', '8만', '조회수 10,049,234회', '2020. 4. 27.', "Rumors are swirling about Kim Jong-un’s location and health. These North Korea experts showed us how they collect information about his secretive regime.\n\nRead the story here: https://nyti.ms/2YaHlqX\nSubscribe: http://bit.ly/U8Ys7n\nMore from The New York Times Video:  http://nytimes.com/video\n----------\nWhether it's reporting on conflicts abroad and political divisions at home, or covering the latest style trends and scientific developments, New York Times video journalists provide a revealing and unforgettable view of the world. It's all the news that's fit to watch.", '10,610']



308
VIDEO INFO:  ['https://www.youtube.com/watch?v=bd0rVu

In [6]:
# 수집한 url을 csv 파일로 저장
df = pd.DataFrame(full_video_data_list, columns=[
    "video_url", "video_title", "video_uploader_name", "video_uploader_url", "likes_count", "views_count", 
    "video_upload_date", "decription_text", "comment_count", "related_videos_list", "comment_data_list"
    ])
df.to_csv("keyword_videos_data_2.csv", index=False)
print("keyword_videos_data_2.csv file saved!")
print(f"error_video_urls ({len(error_video_urls)}개): {error_video_urls}")

keyword_videos_data_2.csv file saved!
error_video_urls (2개): ['https://www.youtube.com/shorts/31rkjIe6YXA', 'https://www.youtube.com/shorts/oS-vn_PwzUE']


---